## Without web search

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
model = init_chat_model(model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        model_provider="groq",
                        temperature = 0,
                        max_retries=3,
                        timeout=60,
                        max_tokens=300
                        )

In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
)

In [4]:
from langchain.messages import HumanMessage

question = HumanMessage(content="How up to date is your training knowledge?")

response = agent.invoke(
    {"messages": [question]}
)

In [5]:
print(response['messages'][-1].content)

My knowledge cutoff is currently August 2024, and my training data goes up to that point. I'll do my best to provide the most accurate information available within my knowledge domain.


## Add web search tool

In [6]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

web_search.invoke("Who is the current mayor of new york?")

{'query': 'Who is the current mayor of new york?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.nyc.gov/mayors-office',
   'title': "Welcome to the Mayor's Office - NYC.gov",
   'content': 'Mayor Eric Adams has served the people of New York City as an NYPD officer, State Senator, Brooklyn Borough President, and now as the 110th Mayor of the',
   'score': 0.9959927,
   'raw_content': None},
  {'url': 'https://www.bbc.com/news/articles/cly4kr8gzr2o',
   'title': 'Zohran Mamdani seals remarkable victory - but real challenges await',
   'content': "Zohran Mamdani, the newly elected mayor of New York City, is notable in many ways. He will become the city's youngest mayor since 1892,",
   'score': 0.98934746,
   'raw_content': None},
  {'url': 'https://www.nyc.gov/mayors-office/mayors-bio',
   'title': "Biography - NYC Mayor's Office",
   'content': 'Mayor Eric Adams has served the people of New York City as an NYPD officer, State Senator, B

In [9]:
agent = create_agent(
    model=model,
    tools=[web_search]
)

question = HumanMessage(content="When Does Mamdani Officially Become Mayor of New York City?")

response = agent.invoke(
    {"messages": [question]}
)

In [11]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content='When Does Mamdani Officially Become Mayor of New York City?', additional_kwargs={}, response_metadata={}, id='da725649-a8c9-4fb5-a339-d71d2ede5eb6'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'nd56fs6v0', 'function': {'arguments': '{"query":"Mamdani becomes Mayor of New York City"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 718, 'total_tokens': 757, 'completion_time': 0.06928184, 'completion_tokens_details': None, 'prompt_time': 0.020233221, 'prompt_tokens_details': None, 'queue_time': 0.383737988, 'total_time': 0.089515061}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b36cb-408b-7bd2-a933-11c4d1f2f8c0-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'Mamdani becomes Mayor of Ne

trace: https://smith.langchain.com/public/59432173-0dd6-49e8-9964-b16be6048426/r